In [29]:
%load_ext autoreload
%autoreload 2
from pangolin import *
from pangolin import inference_jags
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
# very basic inference
x = normal(0,1)
y = normal(x,1)
inf = inference_jags.JAGSInference(niter=10000)
[xs] = inf.sample_flat([x],[y],[np.array(1)])
print(f"{np.mean(xs)=}")

np.mean(xs)=0.4799976664754


In [31]:
print(inference_jags.jags_code(y))

var v4;
model{
v4 ~ dnorm(v2,1/(v3)^2)
}



In [33]:
locs = np.array([1,2,3])
x = vmap(normal,(0,None))(locs, 0.5)
y = vmap(normal,(0,None))(x,1.5)
y_obs = np.array([0.5,1.5,2.5])
inf = inference_jags.JAGSInference(niter=10000)
[xs] = inf.sample_flat([x],[y],[y_obs])
print(f"{xs.shape=}")
print(f"{np.mean(xs,axis=0)=}")

xs.shape=(10000, 3)
np.mean(xs,axis=0)=array([0.95359813, 1.94478386, 2.95696678])


In [4]:
locs = np.array([1,2,3])
x = plate(locs)(lambda loc:
                normal(loc,0.5))
y = plate(x)(lambda xi:
             normal(xi, 1.5))
y_obs = np.array([0.5,1.5,2.5])
inf = inference_jags.JagsInference(niter=10000)
[xs] = inf.sample_flat([x],[y],[y_obs])
print(f"{xs.shape=}")
print(f"{np.mean(xs,axis=0)=}")

var v0[3], v1, v2[3], v3, v4[3];
model{
for (i0 in 1:3){
    v2[i0] ~ dnorm(v0[i0],1/(v1)^2)
}
for (i0 in 1:3){
    v4[i0] ~ dnorm(v2[i0],1/(v3)^2)
}
}

xs.shape=(10000, 3)
np.mean(xs,axis=0)=array([0.95409005, 1.95429682, 2.95165032])


In [5]:
locs = np.array([1,2,3])
x = plate(locs)(lambda loc:
                normal(loc,0.5))
y = plate(N=5)(lambda:
               plate(x)(lambda xi:
                        normal(xi, 1.5)))
y_obs = np.random.randn(5,3)
inf = inference_jags.JagsInference(niter=10000)
[xs] = inf.sample_flat([x],[y],[y_obs])
print(f"{xs.shape=}")
print(f"{np.mean(xs,axis=0)=}")

var v0[3], v1, v2[3], v3, v4[5,3];
model{
for (i0 in 1:3){
    v2[i0] ~ dnorm(v0[i0],1/(v1)^2)
}
for (i0 in 1:5){
    for (i1 in 1:3){
        v4[i0,i1] ~ dnorm(v2[i1],1/(v3)^2)
    }
}
}

xs.shape=(10000, 3)
np.mean(xs,axis=0)=array([0.867326  , 1.11823025, 1.92306224])


In [6]:
x,(y,z) = plate(N=3)\
    (lambda: [x_i := normal(0,1),
              y_i_z_i := plate(N=5)(
                  lambda: [y_ij := normal(x_i,1),
                           z_ij := plate(N=7)(
                               lambda: normal(y_ij,1))
                           ])
              ])
z_obs = np.random.randn(3,5,7)
inf = inference_jags.JagsInference(niter=10000)
[xs] = inf.sample_flat([x],[z],[z_obs])
print(f"{xs.shape=}")
print(f"{np.mean(xs,axis=0)=}")

var v0, v1, v2[3], v3, v4[3,5], v5, v6[3,5,7];
model{
for (i0 in 1:3){
    v2[i0] ~ dnorm(v0,1/(v1)^2)
}
for (i0 in 1:3){
    for (i1 in 1:5){
        v4[i0,i1] ~ dnorm(v2[i0],1/(v3)^2)
    }
}
for (i0 in 1:3){
    for (i1 in 1:5){
        for (i2 in 1:7){
            v6[i0,i1,i2] ~ dnorm(v4[i0,i1],1/(v5)^2)
        }
    }
}
}

xs.shape=(10000, 3)
np.mean(xs,axis=0)=array([-0.02328517, -0.21966823, -0.14661516])


In [7]:
# explicitly construct topics

N = 10 # number of words in each document
M = 4 # number of documents
K = 5 # number of topics
V = 7 # number of words

α = makerv(0.5*np.ones(K))
β = makerv(1*np.ones(V))
θ = plate(N=M)(lambda: dirichlet(α))
φ = plate(N=K)(lambda: dirichlet(β))
z = plate(θ)(lambda θ_m:
             plate(N=N)(lambda:
                        categorical(θ_m)))
assert z.shape == (M,N) # each entry of z is between 0 and K
φz = φ[z]
assert φz.shape == (M,N,V)

w = plate(φz)(lambda φz_m:
              plate(φz_m)(lambda φz_mn:
                          categorical(φz_mn)))

assert w.shape == (M,N)

inf = inference_jags.JagsInference(niter=10000)
[ws] = inf.sample_flat([w],[],[])
print(f"{ws.shape=}")
#print(f"{np.mean(ws,axis=0)=}")

#αs, θs, zs, ws = infer.sample((α,θ,z,w),niter=1)

#viz_upstream(w,[],{θ:'θ',φ:'φ',z:'z',w:'w'})

var v0[7], v1[5,7], v2[5], v3[4,5], v4[4,10], v5[4,10,7], v6[4,10];
model{
for (i0 in 1:5){
    v1[i0,] ~ ddirch(v0)
}
for (i0 in 1:4){
    v3[i0,] ~ ddirch(v2)
}
for (i0 in 1:4){
    for (i1 in 1:10){
        tmp_v4[i0,i1] ~ dcat(v3[i0,])
        v4[i0,i1] <- tmp_v4[i0,i1]-1
    }
}
for (l0 in 1:4){
for (l1 in 1:10){
v5[l0,l1,1:7] <- v1[1+v4[l0,l1],1:7]
}
}
for (i0 in 1:4){
    for (i1 in 1:10){
        tmp_v6[i0,i1] ~ dcat(v5[i0,i1,])
        v6[i0,i1] <- tmp_v6[i0,i1]-1
    }
}
}

ws.shape=(10000, 4, 10)


In [20]:
x = plate(N=3)(lambda:
               categorical([.1,.2,.7]))
#inf_jags = inference_jags.JagsInference(niter=10000)
#[xs] = inf.sample_flat([x],[],[])
calc_jags = calculate.Calculate("jags",niter=10000)
calc_jags.E(x)


var v0[3], v1[3];
model{
for (i0 in 1:3){
    tmp_v1[i0] ~ dcat(v0)
    v1[i0] <- tmp_v1[i0]-1
}
}



array([1.5943, 1.5996, 1.6023])

In [19]:
#inf_numpyro = inference_numpyro.NumpyroInference(niter=10000)
#[xs] = inf.sample_flat([x],[],[])
calc_numpyro = calculate.Calculate("numpyro",niter=10000)
calc_numpyro.E(x)

cond_dist=VMapDist(base_cond_dist=categorical)
rng_key=Traced<ShapedArray(uint32[2])>with<BatchTrace(level=1/0)> with
  val = Array([[2838123479,    8981697],
       [2739061182, 3154801843],
       [2770673021, 1196559342],
       ...,
       [1219400865, 3104058255],
       [2045175387, 1968507671],
       [4249254903,  576000914]], dtype=uint32)
  batch_dim = 0
parent_vals=(array([0.1, 0.2, 0.7]),)


Array([1.5974   , 1.5986999, 1.5886999], dtype=float32)

In [16]:
.2 + .7*2

1.5999999999999999